## Matrix and Vocabulary Construction

In [122]:
import pandas as pd
import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams    
import scipy.sparse as sps
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [3]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

In [4]:
def get_freq(mtuple):
    return mtuple[2]

def get_top3(word, three_array_matrix):
    w_index = vocab[word]
    freqs = list(filter(lambda mt: mt[0] == w_index, three_array_matrix))
    max_freq = sorted(freqs, key=get_freq, reverse=True)
    sorted_indexes_by_freq = list(map(lambda mtuple: mtuple[1], max_freq))
    top3_index = sorted_indexes_by_freq[:3]
    top3_words = list(map(lambda i: index_to_vocab[i], top3_index))
    return top3_words

In [5]:
def expand_query(term, top3_words):
    expanded_query = " OR ".join([term] + top3_words)
    return expanded_query

In [6]:
#Lê e separa as palavras de um texto de determidada notícia, para adicionar ou atualizar o índice 'palavra' do dicionário 
def cataloga_palavras(dicionario, texto, idNoticia):
    palavras = nltk.word_tokenize(texto)
    #mesmo que uma palavra apareça várias vezes no texto,só é necessário adicionar o id dessa notícia uma vez, por isso é utilizado set
    palavras = set(palavras)
    for palavra in palavras:
        palavra = palavra.lower()   #o algoritmo não é case-sensitive
        if palavra in dicionario:
            dicionario[palavra].append(idNoticia)
        else:
            dicionario[palavra] = [idNoticia]

In [7]:
#Retorna os ids de notícias que possuem aquela palavra
def indice_invertido(palavra):
    return dicionario[palavra]

In [31]:
def search(termos):
    termos = termos.split()
    retorno = []
    if len(termos) == 1:   #Caso haja apenas um termo na busca
        return indice_invertido(termos[0].lower())
    for t in termos:
        if t != "OR":
            retorno += indice_invertido(t)
    return set(retorno)   #OR consiste na junção dos resultados, desconsiderando as repetições
    

In [9]:
#Retorna os ids de notícias que possuem aquela palavra
def indice_invertido(palavra):
    return dicionario[palavra]

In [10]:
news = pd.read_csv("estadao_noticias_eleicao.csv", encoding="utf-8")
news.set_index("idNoticia", inplace=True)

In [11]:
news.titulo.fillna("", inplace = True)
news.subTitulo.fillna("", inplace = True)
news.conteudo.fillna("", inplace = True)

In [12]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo

In [123]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))
stopword_ = stopwords.words('portuguese')

filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

tokens_lists = content.apply(lambda text: text.lower().split())
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

matrix, vocab = co_occurrence_matrix(tokens)

In [124]:
index_to_vocab = {v: k for k, v in vocab.items()}

In [125]:
three_array_matrix = list(zip(
    matrix.row, matrix.col, matrix.data))

In [126]:
dicionario = {}

#Cataloga as palavras de todas as notícias da tabela
for index, row in news.iterrows():
    
    #Tratamento de valores NaN na tabela
    titulo = '' if str(row['titulo']) == 'nan' else str(row['titulo'])
    subTitulo = '' if str(row['subTitulo']) == 'nan' else str(row['subTitulo'])
    conteudo = '' if str(row['conteudo']) == 'nan' else str(row['conteudo'])
    
    texto = titulo + " " + subTitulo + " " + conteudo
    
    cataloga_palavras(dicionario, texto, index)

In [128]:
for palavra in ['deputado', 'juiz', 'voto']:
    query = palavra

    top3_words = get_top3(query, three_array_matrix)
    print("Consulta original: " + query)
    print("\nTermos retornados para expansão:", ", ".join(top3_words))

    query_search_result = search(query)
    query_search_result = list(map(str, query_search_result))
    print("\nDocumentos retornados para consulta:", ", ".join(query_search_result))


    expanded_query = expand_query(query, top3_words)
    expanded_query_search_result = search(expanded_query)
    expanded_query_search_result = list(map(str, expanded_query_search_result))
    print("\nDocumentos retornados para consulta expandida:", ", ".join(expanded_query_search_result))


Consulta original: deputado

Termos retornados para expansão: federal, estadual, josé

Documentos retornados para consulta: 2, 7, 21, 22, 23, 25, 26, 26, 33, 40, 41, 43, 44, 45, 46, 51, 58, 60, 61, 62, 77, 77, 78, 81, 83, 85, 89, 91, 93, 94, 104, 110, 110, 111, 113, 116, 120, 120, 122, 124, 126, 131, 131, 146, 147, 150, 150, 154, 159, 165, 165, 166, 171, 172, 176, 177, 177, 180, 181, 183, 190, 200, 201, 213, 217, 238, 245, 264, 268, 268, 280, 283, 284, 293, 303, 305, 306, 312, 313, 318, 323, 326, 330, 334, 346, 347, 348, 352, 356, 362, 366, 370, 371, 375, 396, 406, 413, 429, 430, 434, 438, 442, 444, 450, 467, 470, 474, 484, 488, 493, 494, 498, 502, 507, 509, 511, 526, 526, 532, 536, 542, 550, 567, 587, 603, 615, 616, 621, 624, 627, 629, 638, 639, 648, 649, 652, 657, 670, 671, 672, 673, 674, 676, 690, 697, 703, 704, 708, 713, 716, 717, 721, 721, 724, 725, 728, 732, 733, 734, 735, 736, 749, 753, 755, 759, 762, 774, 786, 787, 788, 793, 797, 797, 802, 804, 807, 809, 816, 817, 823, 837, 851

Consulta original: juiz

Termos retornados para expansão: federal, sérgio, é

Documentos retornados para consulta: 48, 49, 70, 81, 81, 81, 95, 97, 99, 119, 127, 128, 137, 155, 155, 180, 187, 201, 208, 249, 250, 257, 258, 268, 282, 301, 326, 351, 352, 353, 358, 359, 360, 361, 362, 364, 382, 392, 392, 430, 433, 435, 441, 441, 442, 442, 465, 468, 476, 506, 528, 535, 538, 540, 541, 541, 547, 547, 549, 560, 565, 567, 579, 587, 594, 598, 614, 637, 643, 685, 689, 715, 716, 729, 811, 814, 821, 881, 882, 909, 909, 1002, 1156, 1198, 1220, 1233, 1248, 1263, 1280, 1285, 1382, 1391, 1409, 1415, 1421, 1505, 1510, 1556, 1567, 1571, 1577, 1578, 1587, 1615, 1643, 1643, 1670, 1674, 1675, 1692, 1722, 1755, 1766, 1766, 1781, 1786, 1800, 1814, 1816, 1819, 1821, 1823, 1930, 1931, 1948, 2052, 2166, 2167, 2237, 2273, 2302, 2364, 2386, 2395, 2397, 2409, 2409, 2420, 2429, 2525, 2561, 2698, 2699, 2728, 2771, 2771, 2807, 2838, 2865, 2877, 2883, 2896, 2902, 2934, 2951, 2972, 3017, 3026, 3036, 3036, 3078, 3088, 310

Consulta original: voto

Termos retornados para expansão: distrital, dilma, presidente

Documentos retornados para consulta: 56, 69, 82, 101, 110, 158, 160, 176, 179, 180, 217, 232, 270, 279, 282, 306, 347, 366, 396, 400, 410, 414, 425, 467, 501, 514, 526, 529, 532, 542, 543, 553, 576, 591, 624, 631, 632, 657, 671, 672, 699, 702, 708, 709, 716, 718, 732, 748, 770, 771, 800, 813, 822, 829, 831, 835, 838, 839, 848, 851, 854, 861, 863, 864, 865, 878, 890, 910, 938, 941, 942, 944, 952, 971, 1022, 1023, 1030, 1044, 1045, 1063, 1067, 1071, 1072, 1074, 1074, 1076, 1090, 1090, 1099, 1100, 1101, 1106, 1106, 1117, 1129, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1159, 1160, 1161, 1170, 1172, 1173, 1176, 1184, 1187, 1194, 1198, 1202, 1203, 1207, 1212, 1214, 1217, 1218, 1222, 1223, 1224, 1225, 1226, 1229, 1235, 1241, 1243, 1244, 1250, 1254, 1259, 1261, 1268, 1271, 1272, 1282, 1286, 1287, 1288, 1289, 1294, 1295, 1297, 1298, 1300, 1308, 1309, 1313, 1315, 1318, 1330, 1334, 1336, 1337, 1347, 1362

Os termos da expansão possuem um relacionamento forte com o termo pesquisado, como por exemplo 'deputado' e 'federal, estadual'.
Os documentos retornados pela consulta original são mais indicados para as necessidades do usuário, pois com certeza possuem o termo pesquisado, enquanto que os documentos da consulta expandida podem possuir apenas um dos termos da co-ocorrência.
É melhor para o recall, pois adiciona mais opções para serem consultadas.